In [1]:
import pandas as pd
import sympy as sy

In [2]:
inputData = pd.read_csv('M+N Example')

In [3]:
run parseInput.py

In [4]:
run calcIsotopologues.py

In [5]:
def computeM2UValues(M2Solution):
    '''
    Given an M2 gamma, update the M2 output dataframe to include clumped and site-specific delta values
    
    For M+N, can run this same function with 'MN Percent Abundance', 'UM+N', etc. 
    '''
    M2Solution['U Values'] = M2Solution['M2 Percent Abundance'] * M2Solution['UM+2']
    
    #calculate clumped deltas
    clumpedDeltas = [1000*(x/y-1) for x, y in zip(M2Solution['U Values'].values, M2Solution['Stochastic U'].values)]
    clumpedCulled = []
    for i in range(len(clumpedDeltas)):
        if '|' in M2Solution.index[i]:
            clumpedCulled.append(clumpedDeltas[i])
        else:
            clumpedCulled.append('N/A')
      
    #calculate site specific deltas
    deltas = []
    for i, v in M2Solution.iterrows():
        if '|' not in i:
            delta = op.ratioToDelta(v['Composition'],v['U Values'])
            deltas.append(delta)
            
        else:
            deltas.append('N/A')

    M2Solution['Deltas'] = deltas
    M2Solution['Clumped Deltas'] = clumpedCulled
    
    return M2Solution

def rref(B, tol=1e-8, augMatrix = False, debug=False):
    '''
    Adapted from https://gist.github.com/sgsfak/77a1c08ac8a9b0af77393b24e44c9547
    '''
    A = B.copy()
    rows, cols = A.shape
    r = 0
    pivots_pos = []
    row_exchanges = np.arange(rows)
    
    #Don't eliminate final column
    if augMatrix == True:
        rangeTarget = cols - 1
    else: 
        rangeTarget = cols
        
    for c in range(rangeTarget):
        if debug: 
            print("Now at row", r, "and col", c, "with matrix:")
            print(A)

    ## Find the pivot row:
        pivot = np.argmax (np.abs (A[r:rows,c])) + r
        m = np.abs(A[pivot, c])
        
        if debug: 
            print("Found pivot"), m, "in row", pivot
        
        #if m is aproximately 0, say it is really 0 to avoid floating point nonsense
        if m <= tol:
            A[r:rows, c] = np.zeros(rows-r)
            if debug: 
                print("All elements at and below (", r, ",", c, ") are zero.. moving on..")
            
        else:
      ## keep track of bound variables
            pivots_pos.append((r,c))

            if pivot != r:
            ## Swap current row and pivot row
                A[[pivot, r], c:cols] = A[[r, pivot], c:cols]
                row_exchanges[[pivot,r]] = row_exchanges[[r,pivot]]

            if debug: 
                print("Swap row", r, "with row", pivot, "Now:")
                print(A)

            ## Normalize pivot row
            A[r, c:cols] = A[r, c:cols] / A[r, c]
            if debug:
                print("Normalized pivot row")
                print(A)

            ## Eliminate the current column
            v = A[r, c:cols]
            ## Above (before row r):
            if r > 0:
                ridx_above = np.arange(r)
                A[ridx_above, c:cols] = A[ridx_above, c:cols] - np.outer(v, A[ridx_above, c]).T
                
            if debug:
                print("Elimination above performed:")
                print(A)
                
            ## Below (after row r):
            if r < rows-1:
                ridx_below = np.arange(r+1,rows)
                A[ridx_below, c:cols] = A[ridx_below, c:cols] - np.outer(v, A[ridx_below, c]).T
                
                if debug: 
                    print("Elimination below performed:")
                    print(A)
                    
                r += 1
            ## Check if done
            
        if r == rows:
            break;
    return (A, pivots_pos, row_exchanges)

Both to simulate a measurement and to reconstruct the molecule based on measurements, we begin by defining basic information about the experiment. This is included in the top block of the .csv file. It includes our sites, the number of atoms present ("Stoich"), and how fragments sample the sites. It also includes site-specific delta values that will be used to simulate the measurements. 

In [6]:
inputFile = parseInput('C2NO2ExampleClump.csv')
df = pd.DataFrame.from_dict(inputFile['basicInfo'])
fragmentList = inputFile['M1Dict']['Fragment List']
df['Stoich'] = [int(x) for x in list(df['Stoich'])]
df.rename(columns={'atom ID':'site',"element": "IDS", "Stoich": "Number",'Ref Deltas':'deltas'},inplace = True)

In [7]:
df

,site,IDS,Number,Equivalence,70,54,42,deltas
0,C-1,C,1,0.0,1.0,1.0,0.0,25.0
1,C-2,C,1,0.0,1.0,1.0,1.0,-25.0
2,N-3,N,1,0.0,1.0,1.0,1.0,0.0
3,O-4,O,1,0.0,1.0,1.0,1.0,13.0
4,O-5,O,1,0.0,1.0,0.0,0.0,-13.0


In [8]:
siteElements = strSiteElements(df)
siteIsotopes, multinomialCoeff = calculateSetsOfSiteIsotopes(df)
bigA, SN = calcAllIsotopologues(siteIsotopes, multinomialCoeff)
concentrationArray = siteSpecificConcentrations(df)
d = calculateIsotopologueConcentrations(bigA, SN, concentrationArray)

byCondensed = {}
siteElements = strSiteElements(df)
for i, v in d.items():
    condensed = condenseStr(i)
    byCondensed[condensed] = {}
    byCondensed[condensed]['Number'] = v['num']
    byCondensed[condensed]['full'] = i
    byCondensed[condensed]['Conc'] = v['Conc']
    byCondensed[condensed]['Mass'] = np.array(list(map(int,condensed))).sum()
    byCondensed[condensed]['Subs'] = ''.join([uEl(element, int(number)) for element, number in zip(siteElements, condensed)])
    
M0 = {}
M1 = {}
M2 = {}
M3 = {}
M4 = {}

for i, v in byCondensed.items():
    if v['Mass'] == 0:
        M0[i] = v
    if v['Mass'] == 1:
        M1[i] = v
    if v['Mass'] == 2:
        M2[i] = v
    if v['Mass'] == 3:
        M3[i] = v
    if v['Mass'] == 4:
        M4[i] = v
    

In [9]:
inputData

,Unnamed: 0,Abs. Abundance,Rel. Abundance,Adj. Rel. Abundance
0,13C,2.247440e-02,0.022474,0.022474
1,15N,3.676000e-03,0.003676,0.003676
2,17O,7.598000e-04,0.000760,0.000760
3,18O,4.010400e-03,0.004010,0.004010
4,13C13C,1.261957e-04,0.000126,0.000126
5,M1 54,3.635948e-04,0.013934,0.013934
6,M1 54 17O,3.731728e-04,0.014301,0.014301
7,M1 54 15N,3.564566e-03,0.136602,0.136602
8,M1 54 13C,2.179312e-02,0.835163,0.835163
9,M1 42,1.153257e-02,0.441955,0.441955


In [10]:
M2Measurements = {}
for i, v in inputData.iterrows():
    if v['Unnamed: 0'][:2] == 'M2':
        frag = v['Unnamed: 0'].split(' ')[1]
        
        if frag not in M2Measurements:
            M2Measurements[frag] = {}
            
        sub = v['Unnamed: 0'].split(' ')[2]
        if sub == '':
            sub = 'Unsub'
        M2Measurements[frag][sub] = v['Adj. Rel. Abundance']
        
        

In [11]:
pd.set_option("precision", 15)
M2Df = pd.DataFrame.from_dict(M2Measurements)
M2Df.fillna(0, inplace = True)

Next, we input measurement information, including observed M2 values. Although this is just a one-line function, quite a bit is happening under the hood. We'll take a closer look at the output of this function...

In [12]:
M2

{'00002': {'Number': 1,
  'full': '00002',
  'Conc': 0.001895804401698337,
  'Mass': 2,
  'Subs': '18O'},
 '00011': {'Number': 1,
  'full': '00011',
  'Conc': 1.3992534964117162e-07,
  'Mass': 2,
  'Subs': '17O17O'},
 '00020': {'Number': 1,
  'full': '00020',
  'Conc': 0.0019930251402469697,
  'Mass': 2,
  'Subs': '18O'},
 '00101': {'Number': 1,
  'full': '00101',
  'Conc': 1.3365744798559682e-06,
  'Mass': 2,
  'Subs': '15N17O'},
 '00110': {'Number': 1,
  'full': '00110',
  'Conc': 1.3717831287680805e-06,
  'Mass': 2,
  'Subs': '15N17O'},
 '01001': {'Number': 1,
  'full': '01001',
  'Conc': 3.983642784660376e-06,
  'Mass': 2,
  'Subs': '13C17O'},
 '01010': {'Number': 1,
  'full': '01010',
  'Conc': 4.08858170299996e-06,
  'Mass': 2,
  'Subs': '13C17O'},
 '01100': {'Number': 1,
  'full': '01100',
  'Conc': 3.905435274630086e-05,
  'Mass': 2,
  'Subs': '13C15N'},
 '10001': {'Number': 1,
  'full': '10001',
  'Conc': 4.187932158232704e-06,
  'Mass': 2,
  'Subs': '13C17O'},
 '10010': {'Num

In [13]:
#Then defining fragments
#parseInput currently returns floats, not 'x'. Need to fix. 
def zeroToX(y):
    if y == 0:
        return 'x'
    else:
        return int(y)
    
f54 = df['54'].values
f42 = df['42'].values


frag_54 = [zeroToX(z) for z in f54]
frag_42 = [zeroToX(z) for z in f42]

fragments = [frag_54, frag_42]
fragKeys = ['54','42']

In [14]:
UnsubConc = M0['00000']['Conc']

#For each fragment we will observe
for j, fragment in enumerate(fragments):
    #compute the isotopologues present after fragmentation and track their concentrations
    fragmentedDict = {}
    for isotopologue, value in M2.items():
        value['Stochastic U'] = value['Conc'] / UnsubConc
        frag = [fragMult(x,y) for x, y in zip(fragment, isotopologue)]
        newIsotopologue = ''.join(frag)
        M2[isotopologue][fragKeys[j] + ' Identity'] = newIsotopologue
        
        sub = computeSubs(newIsotopologue, siteElements)
        
        if sub == '':
            sub = 'Unsub'
            
        M2[isotopologue][fragKeys[j] + ' Subs'] = sub

In [15]:
M0

{'00000': {'Number': 1,
  'full': '00000',
  'Conc': 0.9696862013627834,
  'Mass': 0,
  'Subs': ''}}

In [16]:
def filterEmptyStr(string):
    if string == '':
        return False
    else:
        return True
    
Isotopologues = pd.DataFrame.from_dict(M2).T
Isotopologues.rename(columns={'Conc':'Stochastic',"Subs": "Composition"},inplace = True)
preciseStrings = []
for i, v in Isotopologues.iterrows():
    Subs = [uEl(element, int(number)) for element, number in zip(siteElements, i)]
    Precise = [x + " " + y for x, y in zip(Subs, df['site']) if x != '']
    output = '   |   '.join(Precise)
    preciseStrings.append(output)
Isotopologues['Precise Identity'] = preciseStrings
Isotopologues.sort_values('Composition',inplace = True)

In [17]:
Isotopologues

,Number,full,Stochastic,Mass,Composition,Stochastic U,54 Identity,54 Subs,42 Identity,42 Subs,Precise Identity
11000,1,11000,0.00012237026985793,2,13C13C,0.0001261957421751,1100x,13C/13C,x100x,13C,13C C-1 | 13C C-2
01100,1,01100,3.90543527463009e-05,2,13C15N,4.027524852e-05,0110x,13C/15N,x110x,13C/15N,13C C-2 | 15N N-3
10100,1,10100,4.1057140066624e-05,2,13C15N,4.234064588e-05,1010x,13C/15N,x010x,15N,13C C-1 | 15N N-3
01001,1,01001,3.98364278466038e-06,2,13C17O,4.108177242351e-06,0100x,13C,x100x,13C,13C C-2 | 17O O-5
01010,1,01010,4.08858170299996e-06,2,13C17O,4.216396703649e-06,0101x,13C/17O,x101x,13C/17O,13C C-2 | 17O O-4
10001,1,10001,4.1879321582327e-06,2,13C17O,4.318852998369e-06,1000x,13C,x000x,Unsub,13C C-1 | 17O O-5
10010,1,10010,4.29825255956406e-06,2,13C17O,4.432622175631e-06,1001x,13C/17O,x001x,17O,13C C-1 | 17O O-4
00101,1,00101,1.33657447985597e-06,2,15N17O,1.3783577388e-06,0010x,15N,x010x,15N,15N N-3 | 17O O-5
00110,1,00110,1.37178312876808e-06,2,15N17O,1.4146670612e-06,0011x,15N/17O,x011x,15N/17O,15N N-3 | 17O O-4
00011,1,00011,1.39925349641172e-07,2,17O17O,1.4429961924231e-07,0001x,17O,x001x,17O,17O O-4 | 17O O-5


In [18]:
M2Df

,54,42
Unsub,0.461185710910699,0.462204494529890
17O,0.000034039150764,0.001079660028414
18O,0.484836260188171,0.484836260188171
15N,0.000325143802349,0.010312970770481
15N/17O,0.000333708887314,0.000333708887314
13C,0.001987870476470,0.030737674718034
13C/17O,0.002040235858829,0.000994614981179
13C/15N,0.019488442864648,0.009500615896516
13C/13C,0.029768587860755,0.000000000000000


In [19]:
CMatrix = []
MeasurementVector = []

closure = np.ones(len(Isotopologues['Number']),dtype = int)
CMatrix.append(closure)
MeasurementVector.append(1)

for fragment in fragKeys:
    IsotopologueFragments = Isotopologues[fragment + ' Subs']
    for sub, v in M2Df[fragment].iteritems():
        c = list(IsotopologueFragments.isin([sub]) * 1)
        CMatrix.append(c)
        MeasurementVector.append(v)

In [20]:
sy.Matrix(CMatrix)

Matrix([
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [21]:
sy.Matrix(MeasurementVector)

Matrix([
[                   1],
[   0.461185710910699],
[   3.403915076412e-5],
[   0.484836260188171],
[0.000325143802348633],
[0.000333708887314251],
[ 0.00198787047647011],
[   0.002040235858829],
[   0.019488442864648],
[  0.0297685878607553],
[    0.46220449452989],
[ 0.00107966002841398],
[   0.484836260188171],
[  0.0103129707704807],
[0.000333708887314251],
[  0.0307376747180345],
[0.000994614981179136],
[ 0.00950061589651587],
[                 0.0]])

We also set up the matrix inversion problem we must solve. We define a "Composition Matrix", where each row corresponds to an individual measurement and each column corresponds to an isotopologue. We also define a "Measurement Vector", where each row gives the result of an individual measurement. The composition matrix takes a vector giving the relative concentrations of each isotopologue in M2 space to the observed measurement; matrix inversion therefore gives the relative concentration of each isotopologue in M2 space.

Additionally, we define "Full Matrix Order" and "Single Fragment Order" vectors. These track what measurement each row of the composition matrix corresponds to. For example, the first row gives closure, the second gives the "18O" substitution of the highest mass fragment, the third gives "13C/13C" substituion of the highest mass fragment, and so forth. Each fragment repeats the same possible substitutions in order. 

We put the information into an augmented matrix to prepare to solve

In [22]:
comp = np.array(CMatrix,dtype=float)
meas = np.array(MeasurementVector,dtype = float)
AugMatrix = np.column_stack((comp, meas))

In [23]:
sy.Matrix(AugMatrix)

Matrix([
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,                  1.0],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,    0.461185710910699],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,    3.403915076412e-5],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,    0.484836260188171],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.000325143802348633],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.000333708887314251],
[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.00198787047647011],
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,    0.002040235858829],
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,    0.019488442864648],
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.0297685878607553],
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,     0.46220449452989],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,  0.001

And solve

In [24]:
solve = rref(AugMatrix, augMatrix = True, debug=False)

In [25]:
sy.Matrix(solve[0])

Matrix([
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,    0.0297685878607552],
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00950061589651588],
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00998782696813208],
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.000969086857279255],
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,  0.000994614981179143],
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00101878361919083],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.00104562087764981],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,  0.000325143802348633],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,   0.00033370888731421],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,   3.40391507641646e-5],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,     0.461185710910699],
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Then we solve, tracking which sums of isotopologues are constrained. The first entry in "sol" gives the solution. The second gives additional information from the "rref" function in M2Module.py.

In [26]:
#Construct augmented matrix
comp = np.array(CMatrix,dtype=float)
meas = np.array(MeasurementVector,dtype = float)
AugMatrix = np.column_stack((comp, meas))

#solve by Gauss Jordan
solve = rref(AugMatrix, augMatrix = True, debug = False)

#Take everything but the final column, which is just the answer
solution = solve[0][:,:-1]

#Check which isotopologues correspond to which measurements in the answer, and explicitly track them
uniqueAnswers = []
stochasticValues = []
composition = []

rank = len(solve[1])
for i in range(len(solution)):
    stoch = 0
    c = None
    
    if i >= rank:
        break
        
    rowIsotopologues = []
    for j in range(len(solution[i])):
        if solution[i][j] == 1:
            rowIsotopologues.append(Isotopologues['Precise Identity'][j])

            stoch += Isotopologues['Stochastic U'][j]
            
            if c == None:
                c = Isotopologues['Composition'][j]
            elif c != Isotopologues['Composition'][j]:
                c = c + " & " + Isotopologues['Composition'][j]
            
    uniqueAnswers.append(rowIsotopologues)
    stochasticValues.append(stoch)
    composition.append(c)
    
#take the measured values
values = solve[0][:rank,-1]

condensed = [' & '.join(x) for x in uniqueAnswers]

#output as dataFrame
output = {}
output['M2 Percent Abundance'] = values
output['Stochastic U'] = stochasticValues
output['Composition'] = composition

dfOutput = pd.DataFrame.from_dict(output)
dfOutput.index = condensed

In [27]:
dfOutput

,M2 Percent Abundance,Stochastic U,Composition
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O


In [28]:
#Manual calculation of UM+2
M218O = dfOutput[dfOutput['Composition'] == '18O']['M2 Percent Abundance'].sum()
UM2 = 4.010400e-03 / M218O

In [29]:
dfOutput['UM+2'] = UM2
dfOutput

,M2 Percent Abundance,Stochastic U,Composition,UM+2
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C,0.004239225010114
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N,0.004239225010114
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N,0.004239225010114
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O,0.004239225010114
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O,0.004239225010114
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O,0.004239225010114
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O,0.004239225010114
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O,0.004239225010114
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O,0.004239225010114
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O,0.004239225010114


In [30]:
computeM2UValues(dfOutput)

,M2 Percent Abundance,Stochastic U,Composition,UM+2,U Values,Deltas,Clumped Deltas
13C C-1 | 13C C-2,0.029768587860755,0.000126195742175,13C13C,0.004239225010114,0.000126195742175,N/A,-2.55351295663786e-12
13C C-2 | 15N N-3,0.009500615896516,0.000040275248520,13C15N,0.004239225010114,0.000040275248520,N/A,4.44089209850063e-13
13C C-1 | 15N N-3,0.009987826968132,0.000042340645880,13C15N,0.004239225010114,0.000042340645880,N/A,-3.33066907387547e-13
13C C-2 | 17O O-5,0.000969086857279,0.000004108177242,13C17O,0.004239225010114,0.000004108177242,N/A,7.94919685631612e-11
13C C-2 | 17O O-4,0.000994614981179,0.000004216396704,13C17O,0.004239225010114,0.000004216396704,N/A,6.43929354282591e-12
13C C-1 | 17O O-5,0.001018783619191,0.000004318852998,13C17O,0.004239225010114,0.000004318852998,N/A,-9.64783808399261e-11
13C C-1 | 17O O-4,0.001045620877650,0.000004432622176,13C17O,0.004239225010114,0.000004432622176,N/A,-4.79616346638068e-11
15N N-3 | 17O O-5,0.000325143802349,0.000001378357739,15N17O,0.004239225010114,0.000001378357739,N/A,0
15N N-3 | 17O O-4,0.000333708887314,0.000001414667061,15N17O,0.004239225010114,0.000001414667061,N/A,-1.2134737659153e-10
17O O-4 | 17O O-5,0.000034039150764,0.000000144299619,17O17O,0.004239225010114,0.000000144299619,N/A,1.31272770431679e-09
